In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

from src.data.yolo_dataset.yolo_dataset import get_train_loader

In [ ]:
def denormalize(img_tensor):
    """
    ImageNet 기준으로 정규화된 Tensor(C,H,W)를 다시 0~255 uint8 이미지(H,W,C)로 되돌린다.
    """
    img = img_tensor.clone().cpu().numpy()  # C,H,W
    mean = np.array([0.485, 0.456, 0.406]).reshape(3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(3, 1, 1)
    img = img * std + mean
    img = np.clip(img, 0, 1)
    img = (img * 255).astype("uint8")
    img = np.transpose(img, (1, 2, 0))  # H,W,C
    return img


def draw_yolo_boxes(image, boxes, color=(0, 255, 0)):
    """
    YOLO 형식 bbox(x_c, y_c, w, h, 0~1)를 픽셀 좌표로 바꿔서 사각형을 그린다.
    """
    h, w, _ = image.shape
    vis = image.copy()
    for x_c, y_c, bw, bh in boxes:
        x1 = int((x_c - bw / 2) * w)
        y1 = int((y_c - bh / 2) * h)
        x2 = int((x_c + bw / 2) * w)
        y2 = int((y_c + bh / 2) * h)
        cv2.rectangle(vis, (x1, y1), (x2, y2), color, 2)
    return vis

In [ ]:
# Train DataLoader 생성
train_loader, dataset = get_train_loader(
    img_dir="datasets/pills/images/train",
    label_dir="datasets/pills/labels/train",
    batch_size=4,
    num_workers=0,
)

# 한 배치 가져오기
images, targets = next(iter(train_loader))

n_show = min(4, len(images))  # 최대 4장 표시

plt.figure(figsize=(4 * n_show, 4))
for i in range(n_show):
    img_t = images[i]  # C,H,W
    boxes = targets[i]["boxes"].cpu().numpy().tolist()

    img_np = denormalize(img_t)
    vis = draw_yolo_boxes(img_np, boxes)

    plt.subplot(1, n_show, i + 1)
    plt.imshow(vis)
    plt.axis("off")
    plt.title(f"Sample {i}")
plt.show()

- 위 셀은 반복 실행할때마다 시각화 결과를 확인할 수 있다.